### Create training dataset for anomaly detection model
In this notebook We are going to create training dataset from node embeddings feature group and register to Hopsworks Feature Store. 
![Training Dataset](./images/create_training_dataset.png)

### Create a connection to hsfs

In [1]:
import hsfs
from hops import hdfs
# Create a connection
connection = hsfs.connection()
# Get the feature store handle for the project's feature store
fs = connection.get_feature_store()

Starting Spark application


ID,Application ID,Kind,State,Spark UI,Driver log
6,application_1651410992845_0007,pyspark,idle,Link,Link


SparkSession available as 'spark'.
Connected. Call `.close()` to terminate connection gracefully.

### Retrieve alert nodes feature group from hsfs

In [2]:
alert_nodes_fg = fs.get_feature_group("alert_nodes_fg", 1)
node_embeddings_fg = fs.get_feature_group("node_embeddings_fg", 1) 

### Prepare training datasets for anomaly detection 
###### In the next notebook we are going to train [gan for anomaly detection](https://arxiv.org/pdf/1905.11034.pdf). Durring training step  we will provide only features of accounts that have never been reported for money laundering behaviour.  But we will disclose previously reported accounts to the model only in evaluation step.   

In [3]:
non_sar_emb_query = node_embeddings_fg.select(["embedding"])\
                                      .join(alert_nodes_fg.select(["is_sar"])\
                                      .filter(alert_nodes_fg.is_sar == 0))

In [4]:
non_sar_emb_query.show(5)

+--------------------+------+
|           embedding|is_sar|
+--------------------+------+
|[-0.2894337177276...|     0|
|[-0.8168580532073...|     0|
|[0.89537668228149...|     0|
|[0.55149841308593...|     0|
|[0.28041338920593...|     0|
+--------------------+------+
only showing top 5 rows

In [5]:
non_sar_emb_query.read().count()

6531

In [6]:
non_sar_td = fs.create_training_dataset(name="gan_non_sar_training_df",
                                       version=1,
                                       data_format="tfrecord",
                                       label=["is_sar"], 
                                       statistics_config={"enabled": False, "histograms": False, "correlations": False, "exact_uniqueness": False}, 
                                       splits={'train': 0.8, 'test': 0.2},
                                       coalesce=True,
                                       description="non sar dataset for gan training")
non_sar_td.save(non_sar_emb_query)

## For testing and evaluation we will include known SAR nodes to measure anomaly score  

In [7]:
non_sar_td = fs.get_training_dataset("gan_non_sar_training_df", 1)
non_sar_test_df = non_sar_td.read(split="test")

In [8]:
sar_emb_query = node_embeddings_fg.select(["embedding"])\
                                  .join(alert_nodes_fg.select(["is_sar"])\
                                  .filter(alert_nodes_fg.is_sar == 1))

In [9]:
sar_df = sar_emb_query.read()
sar_df = sar_df.select(*non_sar_test_df.columns)
eval_df = non_sar_test_df.union(sar_df)
eval_df.cache()
eval_df.show()

+--------------------+------+
|           embedding|is_sar|
+--------------------+------+
|[-0.9998507499694...|     0|
|[-0.9986557960510...|     0|
|[-0.9984421730041...|     0|
|[-0.9970214366912...|     0|
|[-0.9947502613067...|     0|
|[-0.9934816360473...|     0|
|[-0.9908211231231...|     0|
|[-0.9882340431213...|     0|
|[-0.9830579757690...|     0|
|[-0.9823658466339...|     0|
|[-0.9815602302551...|     0|
|[-0.9814951419830...|     0|
|[-0.9812114238739...|     0|
|[-0.9809970855712...|     0|
|[-0.9808425903320...|     0|
|[-0.9780859947204...|     0|
|[-0.9746136665344...|     0|
|[-0.9715352058410...|     0|
|[-0.9711296558380...|     0|
|[-0.9682199954986...|     0|
+--------------------+------+
only showing top 20 rows

In [10]:
non_sar_test_df.count()

1267

In [11]:
sar_df.count()

816

In [12]:
eval_df.count()

2083

In [ ]:
gan_eval_ds = fs.create_training_dataset(name="gan_eval_df",
                                       version=1,
                                       data_format="tfrecord",
                                       label=["is_sar"], 
                                       statistics_config={"enabled": False, "histograms": False, "correlations": False, "exact_uniqueness": False}, 
                                       coalesce = True,
                                       description="evaluation dataset for gan training")
gan_eval_ds.save(eval_df)

## Training dataset provenance
![Training dataset provenance](./images/provenance_td.png)